### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import holidays
import os

## MAVIR import

I resample the data here, since MAVIR has missing values, basic linear interpolation should be enough for the approximately 100 missing values.

In [9]:
mavir = pd.read_csv('mavir_data/mavir.csv', sep=';', parse_dates=['Time'], index_col='Time', date_format="%Y-%m-%d %H:%M:%S")
mavir.columns = ['el_load', 'mavir_pred']
mavir = mavir[:'2023-08-31 23:00:00']
mavir = mavir.resample('1H').mean()
mavir.interpolate(method='linear', inplace=True)
mavir.info()

print('MAVIR MAE:',(mavir['el_load'] - mavir['mavir_pred']).abs().mean())

mavir

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75960 entries, 2015-01-01 00:00:00 to 2023-08-31 23:00:00
Freq: H
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   el_load     75960 non-null  float64
 1   mavir_pred  75960 non-null  float64
dtypes: float64(2)
memory usage: 1.7 MB
MAVIR MAE: 253.013941640337


,el_load,mavir_pred
Time,,
2015-01-01 00:00:00,4270.718,4283.00
2015-01-01 01:00:00,4068.940,4112.50
2015-01-01 02:00:00,3754.788,3847.25
2015-01-01 03:00:00,3510.361,3704.00
2015-01-01 04:00:00,3426.489,3646.75
...,...,...
2023-08-31 19:00:00,5480.864,5657.25
2023-08-31 20:00:00,5139.191,5343.25
2023-08-31 21:00:00,4849.395,5033.50


## OMSZ import

In [ ]:
example = pd.read_csv('omsz_data/53101_Heves_Kékestető.csv', sep=';', parse_dates=['Time'], index_col='Time', date_format="%Y-%m-%d %H:%M:%S")
example.info()

example

Let's check which features have enough datapoints to keep in our dataset

In [ ]:
list_of_files = [f"omsz_data/{file}" for file in os.listdir('omsz_data')]
print(f"Number of files: {len(list_of_files)}")

cols = {'r': 0, 'ta': 0, 'u': 0, 'sg': 0, 'sr': 0, 'we': 0, 'p0': 0, 'f': 0, 'tviz': 0, 'p': 0}

for f in list_of_files:
    df = pd.read_csv(f, sep=';', parse_dates=['Time'], index_col='Time', date_format="%Y-%m-%d %H:%M:%S", nrows=3)
    c = df.columns
    for col in c:
        cols[col] += 1
        if col == 'tviz':
            print(f"Water temperature available in {f.split('/')[1]}, has null values: {df['tviz'].isnull().values.any()}")

print("Number of columns available:", cols)

I'll drop some features that have too few datapoints or are repetitive:
- p: air pressure at instrument level, i'll keep p0, which is the air pressure at sea level
- we: weather code, not possible to aggregate to bigger areas
- sg: seems to have many missing values

tviz: water temperature, not available for most of the stations, but it's available for Balaton via Siófok, and for Velence via Agárd so it's probably worth keeping

In [ ]:
cols_to_drop = ['p', 'we', 'sg', 'tviz']

columns = {
    key: [] for key in cols if key not in cols_to_drop
}
rename_cols = {
    'r': 'prec',
    'ta': 'temp',
    'u': 'rhum',
    'sr': 'grad',
    'p0': 'pres',
    'f': 'wind'
}

tviz_cols = {}

for i, f in enumerate(list_of_files):
    df = pd.read_csv(f, sep=';', parse_dates=['Time'], index_col='Time', date_format="%Y-%m-%d %H:%M:%S")
    for col in df.columns:
        if col not in cols_to_drop:
            columns[col].append(df[col])
        elif col == 'tviz':
            tviz_cols[(f.split('/')[1]).split('_')[0]] = df[col]
            print(f"{f.split('/')[1]} has tviz entries")
            
final_dataframe = mavir.copy().drop(columns=['mavir_pred'])
            
for key, value in columns.items():
    df = pd.DataFrame(index=example.index)
    df = pd.concat([df, *value], axis=1)
    final_dataframe[rename_cols[key]] = df.mean(axis=1, skipna=True)

# for key, value in tviz_cols.items():
#     print(key, value.info()) # based on this info, I'll throw away data recorded in Győr Likócs, since it only has 1 entry

final_dataframe['Vel_tviz'] = tviz_cols['35315']
final_dataframe['Bal_tviz'] = tviz_cols['36100']

# there's only some missing values, mostly in tviz columns, so I'll just interpolate them
final_dataframe.interpolate(method='linear', inplace=True)

final_dataframe.info()

final_dataframe

In [ ]:
holidays_hu = holidays.country_holidays('HU', years=list(range(2015, 2024)))

holidays_hu.get('2023-08-20 01:00:00')

# is it holiday or weekend?
final_dataframe['holiday'] = final_dataframe.index.map(lambda x: 1 if holidays_hu.get(x) else 0)
final_dataframe['weekend'] = final_dataframe.index.map(lambda x: 1 if x.weekday() >= 5 else 0)

final_dataframe

## Covid identifiers

The best resource I could find that has dates is: https://hu.wikipedia.org/wiki/Covid19-koronavírus-járvány_Magyarországon
The official emergency situation was announced on 2020 march 11th.
For the end date, I chose when travel limitations were lifted, which is 2022. march 7. according to https://konzinfo.mfa.gov.hu/covid-19

In [ ]:
final_dataframe['covid'] = 0
final_dataframe.loc['2020-03-11 00:00:00':'2022-03-7 23:00:00', 'covid'] = 1
print(final_dataframe['covid'].value_counts())

final_dataframe['holiday'] = final_dataframe['holiday'].astype(float)
final_dataframe['weekend'] = final_dataframe['weekend'].astype(float)
final_dataframe['covid'] = final_dataframe['covid'].astype(float)

final_dataframe.to_csv('country_data.csv', sep=';')

final_dataframe.info()
final_dataframe